# Exercise - VaR

## Data

This problem uses `weekly` return data from `data/spx_returns_weekly.xlsx`.

Choose any `4` stocks to evaluate below.

For example, 
* `AAPL`
* `META`
* `NVDA`
* `TSLA`

# Section 1: Diversification

## 1.1

Using the full sample, calculate for each series the (unconditional) 

(a) volatility

(b) empirical VaR (.05)

(c) empirical CVaR (.05)

Recall that by **empirical** we refer to the direct quantile estimation. (For example, using `.quantile()` in pandas.

In [18]:
# Question 1.1 Part (a) Code Here

import pandas as pd
import numpy as np

# Load Data
df = pd.read_excel("../data/spx_returns_weekly.xlsx", sheet_name="s&p500 rets")

# Datetime index
df["date"] = pd.to_datetime(df["date"])
df = df.set_index("date").sort_index()

# Tickers
tickers = ["AAPL", "META", "NVDA", "TSLA"]

# Drop NaNs with robust sub-setting and dtype cleanup
cols = [c for c in tickers if c in df.columns]
rets = df[cols].dropna(how="all").apply(pd.to_numeric, errors="coerce")

# Warn if any chosen tickers are missing
missing = sorted(set(tickers) - set(cols))
if missing:
    print("Warning: missing columns =>", missing)

# Unconditional volatility
weekly_vol = rets.std(ddof=1)
annual_vol = weekly_vol * np.sqrt(52)

vol_table = pd.DataFrame({"weekly_vol": weekly_vol, "annualised_vol": annual_vol})

display(vol_table.style.format("{:.4%}"))

print(
    f"Sample: {rets.index.min().date()} to {rets.index.max().date()} "
    f"({rets.shape[0]} weeks)"
)


,weekly_vol,annualised_vol
AAPL,3.8362%,27.6629%
META,4.8722%,35.1336%
NVDA,6.4246%,46.3283%
TSLA,8.1323%,58.6431%


Sample: 2015-01-09 to 2025-05-23 (542 weeks)


In [20]:
# Question 1.1 Part (b) Code Here

alpha = 0.05

# Empirical VaR at 5 percent (historical quantile)
var_signed = rets.quantile(alpha)
var_loss = (-var_signed).clip(lower=0)

var_table = pd.DataFrame(
    {
        "VaR_5pct_signed": var_signed,
        "VaR_5pct_loss": var_loss,
    }
)

# As percentages
print(var_table.applymap(lambda x: f"{x:.2%}"))

     VaR_5pct_signed VaR_5pct_loss
AAPL          -5.64%         5.64%
META          -7.00%         7.00%
NVDA          -8.69%         8.69%
TSLA         -11.74%        11.74%


/var/folders/d8/z5lkh3vn2w51kzd93v0t9kbc0000gn/T/ipykernel_55418/4054416824.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(var_table.applymap(lambda x: f"{x:.2%}"))


In [21]:
# Question 1.1 Part (c) Code Here

alpha = 0.05
var_5 = rets.quantile(alpha)

# Mean return conditional on being below VaR threshold
cvar_signed = rets[rets.le(var_5)].mean()
cvar_loss = (-cvar_signed).clip(lower=0)

cvar_table = pd.DataFrame(
    {
        "VaR_5pct_signed": var_5,
        "CVaR_5pct_signed": cvar_signed,
        "CVaR_5pct_loss": cvar_loss,
    }
)

print(cvar_table.applymap(lambda x: f"{x:.2%}"))

     VaR_5pct_signed CVaR_5pct_signed CVaR_5pct_loss
AAPL          -5.64%           -8.31%          8.31%
META          -7.00%          -10.32%         10.32%
NVDA          -8.69%          -11.65%         11.65%
TSLA         -11.74%          -14.78%         14.78%


/var/folders/d8/z5lkh3vn2w51kzd93v0t9kbc0000gn/T/ipykernel_55418/1531063214.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(cvar_table.applymap(lambda x: f"{x:.2%}"))


-----

## 1.2
(a) Form an equally-weighted portfolio of the investments.

In [23]:
# Question 1.2 Part (a) Code Here

n_assets = rets.shape[1]
weights = np.repeat(1 / n_assets, n_assets)

# Compute portfolio returns
port_rets = rets.dot(weights)

# Inspect Summary
print(port_rets.describe()[["mean", "std", "min", "max"]])


mean    0.007769
std     0.043758
min    -0.164886
max     0.155550
dtype: float64


(b) Calculate the statistics of `1.1` for this portfolio, and compare the results to the individual return statistics. 

In [25]:
# Question 1.2 Part (b) Code Here (clean combined output)

# Portfolio statistics
alpha = 0.05
port_weekly_vol = port_rets.std(ddof=1)
port_annual_vol = port_weekly_vol * np.sqrt(52)
port_var = port_rets.quantile(alpha)
port_cvar = port_rets[port_rets <= port_var].mean()

# Combine individual stats into one table
individual_stats = pd.concat(
    [vol_table, var_table["VaR_5pct_signed"], cvar_table["CVaR_5pct_signed"]],
    axis=1
)

# Add portfolio row
portfolio_stats = pd.DataFrame(
    {
        "weekly_vol": [port_weekly_vol],
        "annualised_vol": [port_annual_vol],
        "VaR_5pct_signed": [port_var],
        "CVaR_5pct_signed": [port_cvar],
    },
    index=["Portfolio"],
)

# Append portfolio row to the bottom
compare = pd.concat([individual_stats, portfolio_stats])

# Display as clean percentages
print(compare.applymap(lambda x: f"{x:.2%}"))


          weekly_vol annualised_vol VaR_5pct_signed CVaR_5pct_signed
AAPL           3.84%         27.66%          -5.64%           -8.31%
META           4.87%         35.13%          -7.00%          -10.32%
NVDA           6.42%         46.33%          -8.69%          -11.65%
TSLA           8.13%         58.64%         -11.74%          -14.78%
Portfolio      4.38%         31.55%          -6.19%           -8.50%


/var/folders/d8/z5lkh3vn2w51kzd93v0t9kbc0000gn/T/ipykernel_55418/2714080794.py:31: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(compare.applymap(lambda x: f"{x:.2%}"))


(c) What do you find? What is driving this result?

The equally weighted portfolio shows lower volatility, VaR, and CVaR than the individual stocks. 

This confirms the effect of diversification: combining imperfectly correlated assets reduces total portfolio risk, even though each stock remains risky on its own.

The result is driven by less-than-perfect correlations among the assets’ returns. 

When one stock performs poorly, others may perform better, so their losses do not occur simultaneously. 

As a result, portfolio return fluctuations are smoother, leading to a smaller standard deviation and smaller tail losses (VaR and cVaR).

-----

## 1.3
(a) Re-calculate `1.2`, but this time drop your most volatile asset, and replace the portion it was getting with 0. 

You could imagine we're replacing the most volatile asset with a negligibly small risk-free rate.

In [26]:
# Question 1.3 Part (a) Code Here

# Identify most volatile asset
most_volatile = vol_table["weekly_vol"].idxmax()
print(f"Most volatile asset dropped: {most_volatile}")

# Set that column's returns to zero
rets_adj = rets.copy()
rets_adj[most_volatile] = 0.0

# Keep the same equal weights (including the dropped one)
n_assets = rets_adj.shape[1]
weights = np.repeat(1 / n_assets, n_assets)

# Compute adjusted portfolio returns
port_rets_adj = rets_adj.dot(weights)

# Recalculate portfolio statistics
alpha = 0.05
port_weekly_vol_adj = port_rets_adj.std(ddof=1)
port_annual_vol_adj = port_weekly_vol_adj * np.sqrt(52)
port_var_adj = port_rets_adj.quantile(alpha)
port_cvar_adj = port_rets_adj[port_rets_adj <= port_var_adj].mean()

# Combine results for comparison
compare_adj = pd.DataFrame(
    {
        "weekly_vol": [port_weekly_vol, port_weekly_vol_adj],
        "annualised_vol": [port_annual_vol, port_annual_vol_adj],
        "VaR_5pct_signed": [port_var, port_var_adj],
        "CVaR_5pct_signed": [port_cvar, port_cvar_adj],
    },
    index=["Original Portfolio", "Dropped Most Volatile"],
)

print(compare_adj.applymap(lambda x: f"{x:.2%}"))


Most volatile asset dropped: TSLA
                      weekly_vol annualised_vol VaR_5pct_signed  \
Original Portfolio         4.38%         31.55%          -6.19%   
Dropped Most Volatile      3.03%         21.84%          -4.25%   

                      CVaR_5pct_signed  
Original Portfolio              -8.50%  
Dropped Most Volatile           -6.05%  


/var/folders/d8/z5lkh3vn2w51kzd93v0t9kbc0000gn/T/ipykernel_55418/3084603792.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(compare_adj.applymap(lambda x: f"{x:.2%}"))


(b) In comparing the answer here to 1.2, how much risk is your most volatile asset adding to the portfolio? 

By comparing the two portfolios, the most volatile asset (TSLA) adds roughly 1.35 percentage points of weekly volatility to the portfolio (4.38% − 3.03%), or about 9.7 percentage points annualised (31.55% − 21.84%).

Similarly, the 5% VaR worsened from −4.25% to −6.19%, and the CVaR from −6.05% to −8.50% when TSLA was included.

This shows that TSLA contributes meaningfully to the portfolio’s total risk — both overall volatility and tail losses, due to its large individual variance and imperfect diversification with the other assets. 

Even though diversification offsets some of its volatility, TSLA’s high standalone risk still increases the portfolio’s downside exposure substantially.

(c) Is this in line with the amount of risk we measured in the stand-alone risk-assessment of `1.1`?

Yes, this is consistent with the stand-alone risk assessment in 1.1. 

TSLA had the highest individual volatility (about 8.1% weekly and 58.6% annualised) and the deepest tail losses (VaR ≈ −11.7% and CVaR ≈ −14.8%). W

hen included in the portfolio, it increased total volatility and downside risk by roughly the same proportion as its stand-alone risk suggested.

Although diversification reduced part of the impact, TSLA’s very high variance and strong co-movement with the other growth stocks still raised the portfolio’s overall risk. 

The results are therefore in line with the individual metrics from 1.1, confirming that TSLA is the main contributor to total portfolio volatility and tail loss.

-----

# Section 2: Dynamic Measures

## 2.1 

Let's measure the **conditional** statistics of the equally-weighted portfolio of `1.2`, as of the end of the sample.

Note:
- Suppose we can approximate that the daily mean return is zero.
- In this setup, we are using a forecasted volatility, $\sigma_t$ to estimate the VaR return we would have estimated at the end of $t-1$ in prediction of time $t$.

(a) Volatility

For each security, calculate the **rolling** volatility series, $\sigma_t$, with a window of $m=26$.

The value at $\sigma_t$ in the notes denotes the estimate using data through time $t-1$, and thus (potentially) predicting the volatility at $\sigma_{t}$. 

In [27]:
# Question 2.1 Part (a) Code Here

# rolling window length (weeks)
m = 26

# Rolling volatility for each individual stock
rolling_vol = rets.rolling(window=m).std(ddof=1)

# Rolling portfolio volatility (using equal weights)
n_assets = rets.shape[1]
weights = np.repeat(1 / n_assets, n_assets)
port_rets = rets.dot(weights)
rolling_vol_port = port_rets.rolling(window=m).std(ddof=1)

# Quick check
print("Last few rolling volatilities (weekly):")
print(rolling_vol_port.tail())

# Final forecasted volatilities
latest_vol = pd.concat(
    [rolling_vol.iloc[-1], pd.Series({"Portfolio": rolling_vol_port.iloc[-1]})]
)
print("\nFinal rolling volatilities (σ_t):")
print(latest_vol.apply(lambda x: f"{x:.2%}"))


Last few rolling volatilities (weekly):
date
2025-04-25    0.053597
2025-05-02    0.053211
2025-05-09    0.048479
2025-05-16    0.053749
2025-05-23    0.054049
dtype: float64

Final rolling volatilities (σ_t):
AAPL         5.15%
META         5.80%
NVDA         8.30%
TSLA         8.30%
Portfolio    5.40%
dtype: object


(b) VaR

Calculate the **normal VaR** and **normal CVaR** for $q=.05$ and $\tau=1$ as of the end of the sample.Use the approximation, $\texttt{z}_{.05} = -1.65$.

In [28]:
# Question 2.1 Part (b) Code Here

# Parameters
z_05 = -1.65
alpha = 0.05

# Use the final rolling portfolio volatility as σ_t
sigma_t = rolling_vol_port.iloc[-1]

# Normal VaR (1-week, assuming mean ≈ 0)
var_norm = z_05 * sigma_t

# Normal CVaR (expected shortfall under normality)
# Formula: CVaR = (φ(z_q) / α) * σ, where φ is standard normal pdf
phi_z = (1 / np.sqrt(2 * np.pi)) * np.exp(-0.5 * z_05**2)
cvar_norm = -sigma_t * (phi_z / alpha)

# Display results
print(f"Final σ_t (weekly): {sigma_t:.4%}")
print(f"Normal VaR (5%): {var_norm:.4%}")
print(f"Normal CVaR (5%): {cvar_norm:.4%}")


Final σ_t (weekly): 5.4049%
Normal VaR (5%): -8.9180%
Normal CVaR (5%): -11.0546%


(c) Conclude and Compare
Report
* volatility (annualized).
* normal VaR (.05)
* normal CVaR (.05)

How do these compare to the answers in `1.2`?

In [29]:
# Question 2.1 Part (c) Code Here

# Calculate annualised volatility from σ_t
port_vol_annual_cond = sigma_t * np.sqrt(52)

# Create comparison table
compare_dynamic = pd.DataFrame({
    "Volatility_weekly": [sigma_t, port_weekly_vol],
    "Volatility_annual": [port_vol_annual_cond, port_annual_vol],
    "VaR_5pct": [var_norm, port_var],
    "CVaR_5pct": [cvar_norm, port_cvar]
}, index=["Conditional (σ_t from 2a)", "Unconditional (from 1.2)"])

# Display as percentages
print(compare_dynamic.applymap(lambda x: f"{x:.2%}"))

# Summary comment
print(
    "\nInterpretation:\n"
    "The conditional (rolling) volatility and corresponding VaR/CVaR are higher than "
    "the unconditional values from 1.2. This indicates that recent market conditions "
    "have been more volatile, so a dynamic model predicts larger potential losses."
)


                          Volatility_weekly Volatility_annual VaR_5pct  \
Conditional (σ_t from 2a)             5.40%            38.98%   -8.92%   
Unconditional (from 1.2)              4.38%            31.55%   -6.19%   

                          CVaR_5pct  
Conditional (σ_t from 2a)   -11.05%  
Unconditional (from 1.2)     -8.50%  

Interpretation:
The conditional (rolling) volatility and corresponding VaR/CVaR are higher than the unconditional values from 1.2. This indicates that recent market conditions have been more volatile, so a dynamic model predicts larger potential losses.


/var/folders/d8/z5lkh3vn2w51kzd93v0t9kbc0000gn/T/ipykernel_55418/81550451.py:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(compare_dynamic.applymap(lambda x: f"{x:.2%}"))


-----

## 2.2

Backtest the VaR using the **hit test**. 

Namely, check how many times the realized return at $t$ was smaller than the VaR return calculated using $\sigma_t$, (where again remember the notation in the notes uses $\sigma_t$ as a vol based on data through $t-1$.)

Report the percentage of "hits" using both the

(a) expanding volatility

(b) rolling volatility

In [30]:
# Question 2.2 Part (a) Code Here

alpha = 0.05
z_05 = -1.65

# Compute expanding (cumulative) volatility through t-1
expanding_vol = rets.dot(np.repeat(1 / rets.shape[1], rets.shape[1])) \
    .expanding(min_periods=26).std(ddof=1)

# Align volatility one period back to represent the forecast at t-1
sigma_forecast = expanding_vol.shift(1)

# Compute predicted VaR_t = z * sigma_{t-1}
var_pred = z_05 * sigma_forecast

# Actual portfolio returns (realised)
port_rets = rets.dot(np.repeat(1 / rets.shape[1], rets.shape[1]))

# Hit = 1 if actual return < predicted VaR
hits = (port_rets < var_pred).astype(int)

# Percentage of violations ("hits")
hit_rate = hits.mean() * 100

print(f"Percentage of VaR hits (expanding volatility): {hit_rate:.2f}%")

# Optional: check the expected level under correct coverage
print(f"Expected under 5% VaR: {alpha * 100:.2f}%")


Percentage of VaR hits (expanding volatility): 4.98%
Expected under 5% VaR: 5.00%


In [32]:
# Question 2.2 Part (b) Code Here

alpha = 0.05
z_05 = -1.65
m = 26  # rolling window length (weeks)

# Rolling volatility (σ_t based on previous 26 weeks)
rolling_vol = port_rets.rolling(window=m).std(ddof=1)

# Shift one period back so σ_{t-1} predicts risk for time t
sigma_forecast_roll = rolling_vol.shift(1)

# Predicted VaR_t = z * σ_{t-1}
var_pred_roll = z_05 * sigma_forecast_roll

# Hit = 1 if realised return < predicted VaR
hits_roll = (port_rets < var_pred_roll).astype(int)

# Percentage of violations ("hits")
hit_rate_roll = hits_roll.mean() * 100

print(f"Percentage of VaR hits (rolling volatility): {hit_rate_roll:.2f}%")
print(f"Expected under 5% VaR: {alpha * 100:.2f}%")


Percentage of VaR hits (rolling volatility): 4.06%
Expected under 5% VaR: 5.00%
